In [6]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import requests
from urllib.parse import urljoin

base_url = "https://bulletins.nyu.edu"

programs = pd.read_csv("program_urls.csv")

REQ_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    )
}

all_courses = []

for idx, program in tqdm(programs.iterrows(), total=len(programs)):
    # Skip if already scraped
    if str(program.get("visited", "False")) == "True":
        continue

    program_href = program["href"]
    url = base_url + program_href + "/#curriculumtext"
    print("Visiting:", url)

    try:
        resp = requests.get(url, headers=REQ_HEADERS, timeout=10)
        resp.raise_for_status()
    except Exception as e:
        print(f"Error fetching program page {url}: {e}")
        continue

    soup = BeautifulSoup(resp.content, "html.parser")

    courselist = soup.find("table", class_="sc_courselist")
    if not courselist:
        print("No courselist table found for", url)
        continue

    tbody = courselist.find("tbody")
    if not tbody:
        print("No <tbody> found in courselist for", url)
        continue

    rows = tbody.find_all("tr")

    req_num = 0
    for tr in rows:
        row_text = tr.get_text(" ", strip=True)
        tr_classes = tr.get("class") or []

        # --- Skip area headers and subheaders (organizational only) ---
        if "areaheader" in tr_classes or "areasubheader" in tr_classes:
            continue

        # --- Handle comment rows that indicate requirements ---
        comment_span = tr.find("span", class_="courselistcomment")
        if comment_span:
            comment_classes = comment_span.get("class") or []
            if "areaheader" not in comment_classes and "areasubheader" not in comment_classes:
                comment_text = comment_span.get_text(" ", strip=True)
                # Skip if it's just a totals row or empty
                if "Total Credits" in comment_text or "listsum" in tr_classes:
                    continue
                # If it's a requirement comment (not "Select one of the following"), increment req_num
                if comment_text and "Select one of the following" not in comment_text:
                    # Check if this row has a course link - if not, it's a standalone requirement
                    a = tr.find("a", class_="bubblelink code")
                    if not a:
                        req_num += 1

                        # Extract credits from the row if available
                        tds = tr.find_all("td")
                        credits_table = ""
                        last_td = tds[-1] if tds else None
                        if last_td is not None and "hourscol" in (last_td.get("class") or []):
                            credits_table = last_td.get_text(strip=True)

                        # Save the comment row as a requirement entry
                        all_courses.append({
                            "program_name": program["name"],
                            "req_num": req_num,
                            "course_code": "",  # No course code for comment rows
                            "title": comment_text,
                            "credits": credits_table,
                            "offered": "",
                            "description": "",
                            "grading": "",
                            "repeatable": "",
                            "prerequisites": "",
                            "extra_blocks": "",
                            "detail_url": "",
                        })
                        continue

        # --- Handle "Select one of the following" rows ---
        if "Select one of the following" in row_text:
            # Every time this appears, count it as a new requirement
            req_num += 1
            continue  # usually these rows don't have course links

        a = tr.find("a", class_="bubblelink code")
        if not a:
            continue  # skip non-course rows

        # --- Check if course is indented (part of "Select one" group) ---
        is_indented = False
        blockindent = a.find_parent("div", class_="blockindent")
        if blockindent:
            is_indented = True

        # --- Increment req_num for new requirements ---
        # Don't increment if: it's an "orclass" row or it's indented (part of "Select one" group)
        if "orclass" not in tr_classes and not is_indented:
            req_num += 1

        course_code_table = a.get_text(strip=True)
        course_href = a.get("href")
        course_url = urljoin(base_url, course_href)

        tds = tr.find_all("td")
        title_table = tds[1].get_text(" ", strip=True) if len(tds) > 1 else ""

        credits_table = ""
        last_td = tds[-1] if tds else None
        if last_td is not None and "hourscol" in (last_td.get("class") or []):
            credits_table = last_td.get_text(strip=True)

        # ----- Fetch and parse course detail page -----
        course_code = course_code_table
        title = title_table
        credits = credits_table
        offered = ""
        description = ""
        grading = ""
        repeatable = ""
        prerequisites = ""   # initialize safely
        extra_blocks = []

        try:
            detail_resp = requests.get(course_url, headers=REQ_HEADERS, timeout=10)
            detail_resp.raise_for_status()
            detail_soup = BeautifulSoup(detail_resp.content, "html.parser")

            courseblock = detail_soup.find("div", class_="courseblock")
            if courseblock:
                # direct child divs inside .courseblock
                child_divs = [div for div in courseblock.find_all("div", recursive=False)]

                # 0: code, title, credits
                if len(child_divs) >= 1:
                    spans = child_divs[0].find_all("span")
                    if len(spans) >= 1:
                        course_code = spans[0].get_text(" ", strip=True)
                    if len(spans) >= 2:
                        title = spans[1].get_text(" ", strip=True)
                    if len(spans) >= 3:
                        credits = spans[2].get_text(" ", strip=True)

                # 1: Typically offered
                if len(child_divs) >= 2:
                    offered_div = child_divs[1]
                    offered_span = offered_div.select_one(".detail-typically_offered")
                    if offered_span:
                        label_span = offered_span.find("span", class_="label")
                        if label_span and label_span.next_sibling:
                            offered = str(label_span.next_sibling).strip()
                        else:
                            raw = offered_span.get_text(" ", strip=True)
                            offered = raw.replace("Typically offered", "").strip(" :")
                    else:
                        offered = offered_div.get_text(" ", strip=True)

                # 2: Description
                if len(child_divs) >= 3:
                    description_div = child_divs[2]
                    description = description_div.get_text(" ", strip=True).replace("\n", "")

                # 3: Grading
                if len(child_divs) >= 4:
                    grading_div = child_divs[3]
                    grading_span = grading_div.select_one(".detail-grading")
                    if grading_span:
                        label = grading_span.find("span", class_="label")
                        if label and label.next_sibling:
                            grading = str(label.next_sibling).strip()
                        else:
                            raw = grading_span.get_text(" ", strip=True)
                            grading = raw.replace("Grading:", "").strip()

                # 4: Repeatable for Credit
                if len(child_divs) >= 5:
                    repeat_text = child_divs[4].get_text(" ", strip=True)
                    if ":" in repeat_text:
                        repeatable = repeat_text.split(":", 1)[1].strip()
                    else:
                        repeatable = repeat_text.strip()

                # 5: Prerequisites
                if len(child_divs) >= 6:
                    prereq_text = child_divs[5].get_text(" ", strip=True)
                    if ":" in prereq_text:
                        prerequisites = prereq_text.split(":", 1)[1].strip()
                    else:
                        prerequisites = prereq_text.strip()

                # 6+: Extra blocks
                if len(child_divs) > 6:
                    for extra_div in child_divs[6:]:
                        text = extra_div.get_text(" ", strip=True)
                        if text:
                            extra_blocks.append(text)

        except Exception as e:
            print(f"Error fetching/parsing details for {course_code_table} at {course_url}: {e}")

        all_courses.append({
            "program_name": program["name"],
            "req_num": req_num,
            "course_code": course_code,
            "title": title,
            "credits": credits,
            "offered": offered,
            "description": description,
            "grading": grading,
            "repeatable": repeatable,
            "prerequisites": prerequisites,
            "extra_blocks": " || ".join(extra_blocks),
            "detail_url": course_url,
        })

    # mark this program as visited so we can resume later
    programs.loc[idx, "visited"] = "True"

# write out
df = pd.DataFrame(all_courses)
df.to_csv("courses_detailed.csv", index=False)
programs.to_csv("program_urls.csv", index=False)
print("courses_detailed.csv written")


  0%|          | 0/164 [00:00<?, ?it/s]

Visiting: https://bulletins.nyu.edu/undergraduate/arts-science/programs/advanced-mathematical-methods-minor//#curriculumtext


/tmp/ipython-input-1082577555.py:237: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  programs.loc[idx, "visited"] = "True"
  1%|          | 1/164 [00:04<12:11,  4.49s/it]

Visiting: https://bulletins.nyu.edu/undergraduate/arts-science/programs/africana-studies-ba//#curriculumtext


  1%|          | 2/164 [00:07<10:29,  3.88s/it]

...

Visiting: https://bulletins.nyu.edu/undergraduate/arts-science/programs/urban-studies-sociology-ba//#curriculumtext


 99%|█████████▉| 163/164 [12:10<00:02,  2.77s/it]

Visiting: https://bulletins.nyu.edu/undergraduate/arts-science/programs/web-programming-applications-minor//#curriculumtext


100%|██████████| 164/164 [12:12<00:00,  4.47s/it]

courses_detailed.csv written
